In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys; sys.path.insert(0, '..')

In [ ]:
import pandas as pd
import numpy as np
import geopandas as gpd
from qr_code import QR_CODE, get_compact

In [ ]:
def print_grid(arr):
    black_square = "█"       # U+2B1B - BLACK LARGE SQUARE
    white_square = " "       # U+2B1C - WHITE LARGE SQUARE
    return [''.join(np.where(row, black_square, white_square)) for row in arr]

## Load File

See what cells of USNG intersect with Camberville

In [ ]:
df = pd.read_csv("../data/USNG_joined.csv")

In [ ]:
df['USNGCoord']

In [ ]:
df['easting'] = df['USNGCoord'].str.split().str[-2].astype('int')
df['northing'] = df['USNGCoord'].str.split().str[-1].astype('int')

In [ ]:
east_min, east_max = df['easting'].min(), df['easting'].max()
north_min, north_max = df['northing'].min(), df['northing'].max()

## Filter full data

In [ ]:
# gdf_full = gpd.read_file('../data/US_National_Grid_HFL_V_7334664624376357600.zip')
gdf_full = gpd.read_file('../data/USNG_reprojected.gpkg')

In [ ]:
gdf_full["fill"] = False
gdf_full["isqr"] = False

In [ ]:
gdf_full['easting'] = gdf_full['USNGCoord'].str.split().str[-2].astype('int')
gdf_full['northing'] = gdf_full['USNGCoord'].str.split().str[-1].astype('int')

In [ ]:
gdf_filtered = gdf_full[
    (gdf_full['easting'] >= east_min) & (gdf_full['easting'] <= east_max) &
    (gdf_full['northing'] >= north_min) & (gdf_full['northing'] <= north_max)
]

In [ ]:
gdf = gpd.GeoDataFrame(gdf_filtered)

In [ ]:
gdf = gdf.sort_values(
    by=['easting', 'northing'],
    ascending=[True, False])

## Reshape

In [ ]:
h = len(gdf['northing'].unique())
w = len(gdf['easting'].unique())

In [ ]:
fill_array = gdf['fill'].values.reshape(w, h).T
isqr_array = gdf['isqr'].values.reshape(w, h).T

### Place QR inside grid

In [ ]:
qr_arr = get_compact(QR_CODE)
qr_arr.shape

In [ ]:
# make array same size as qr_arr but all True
qr_size_arr = np.ones_like(qr_arr, dtype=bool)

In [ ]:
start_x, start_y = 30, 40

In [ ]:
# add qr code to fill_array
start_row, start_col = start_x, start_y
fill_array[start_row:start_row+25, start_col:start_col+25] = qr_arr
fill_array[start_row:start_row+25, start_col:start_col+25] = qr_arr

# add qr_size_arr to isqr_array
start_row, start_col = start_x, start_y  # example coordinates
isqr_array[start_row:start_row+25, start_col:start_col+25] = qr_size_arr
isqr_array[start_row:start_row+25, start_col:start_col+25] = qr_size_arr

In [ ]:
# print_grid(fill_array)

### Unshape again

In [ ]:
gdf['fill'] = fill_array.T.flatten()
gdf['isqr'] = isqr_array.T.flatten()

In [ ]:
gdf.to_file('../out/USNG_QR.gpkg', driver='GeoJSON')